In [5]:
import kfp
import kfp.dsl as dsl
from kfp import components

from kubeflow.katib import ApiClient
from kubeflow.katib import V1beta1ExperimentSpec
from kubeflow.katib import V1beta1AlgorithmSpec
from kubeflow.katib import V1beta1ObjectiveSpec
from kubeflow.katib import V1beta1ParameterSpec
from kubeflow.katib import V1beta1FeasibleSpace,V1beta1EarlyStoppingSetting
from kubeflow.katib import V1beta1TrialTemplate
from kubeflow.katib import V1beta1TrialParameterSpec

In [6]:
experiment_name = "fashion-mnist-katib-experiment"
experiment_namespace = "kubeflow"

# Trial count specification.
max_trial_count = 6
parallel_trial_count = 6
max_failed_trial_count = 3


# Objective specification.
objective=V1beta1ObjectiveSpec(
    type="maximize",
    goal= 0.99,
    objective_metric_name="Test-accuracy",
    additional_metric_names=[
        "Train-accuracy",
        "Validation-accuracy"
    ]
)

# Algorithm specification.
algorithm=V1beta1AlgorithmSpec(
    algorithm_name="random",
)


# Early Stopping specification.
# early_stopping=V1beta1EarlyStoppingSpec(
#     algorithm_name="medianstop",
#     algorithm_settings=[
#         V1beta1EarlyStoppingSetting(
#             name="min_trials_required",
#             value="2"
#         )
#     ]
# )


# Experiment search space.
# In this example we tune learning rate, number of layer and optimizer.
# Learning rate has bad feasible space to show more early stopped Trials.
parameters=[
    V1beta1ParameterSpec(
        name="learningRate",
        parameter_type="double",
        feasible_space=V1beta1FeasibleSpace(
            min="0.01",
            max="0.3"
        ),
    ),
    V1beta1ParameterSpec(
        name="numberLayers",
        parameter_type="int",
        feasible_space=V1beta1FeasibleSpace(
            min="1",
            max="5"
        ),
    ),
    V1beta1ParameterSpec(
        name="optimizer",
        parameter_type="categorical",
        feasible_space=V1beta1FeasibleSpace(
            list=[
                "sgd",
                "adam",
                "ftrl"
            ]
        ),
    ),
    V1beta1ParameterSpec(
        name="dropOut",
        parameter_type="double",
        feasible_space=V1beta1FeasibleSpace(
            min="0.1",
            max="0.8"
        ),
    ),
    V1beta1ParameterSpec(
        name="batchSize",
        parameter_type="int",
        feasible_space=V1beta1FeasibleSpace(
            min="8",
            max="32"
        ),
    ),
    V1beta1ParameterSpec(
        name="numDims",
        parameter_type="int",
        feasible_space=V1beta1FeasibleSpace(
            min="1",
            max="5"
        ),
    ),
]

In [7]:
# JSON template specification for the Trial's Worker Kubernetes Job.
trial_spec={
    "apiVersion": "batch/v1",
    "kind": "Job",
    "spec": {
        "template": {
            "metadata": {
                "annotations": {
                    "sidecar.istio.io/inject": "false"
                }
            },
            "spec": {
                "imagePullSecrets": [{
                    "name": "registry-credentials"
                }],
                "containers": [
                    {
                        "name": "training-container",
                        "image": "katib-models",
                        "command": [
                            "python",
                            "/opt/model.py",
                            "--tf-learning-rate=${trialParameters.learningRate}",
                            "--tf-num-layers=${trialParameters.numberLayers}",
                            "--tf-optimizer=${trialParameters.optimizer}",
                            "--tf-dropout=${trialParameters.dropOut}",
                            "--tf-batch-size=${trialParameters.batchSize}",
                            "--tf-num-dims=${trialParameters.numDims}",
                            "--tf-epochs=32"
                        ]
                    }
                ],
                "restartPolicy": "Never"
            }
        }
    }
}

# Configure parameters for the Trial template.
# We set the retain parameter to "True" to not clean-up the Trial Job's Kubernetes Pods.
trial_template=V1beta1TrialTemplate(
    retain=True,
    primary_container_name="training-container",
    trial_parameters=[
        V1beta1TrialParameterSpec(
            name="learningRate",
            description="Learning rate for the training model",
            reference="learningRate"
        ),
        V1beta1TrialParameterSpec(
            name="numberLayers",
            description="Number of training model layers",
            reference="numberLayers"
        ),
        V1beta1TrialParameterSpec(
            name="optimizer",
            description="Training model optimizer (sdg, adam or ftrl)",
            reference="optimizer"
        ),
        V1beta1TrialParameterSpec(
            name="dropOut",
            description="drop out",
            reference="dropOut"
        ),
        V1beta1TrialParameterSpec(
            name="batchSize",
            description="batch size",
            reference="batchSize"
        ),
        V1beta1TrialParameterSpec(
            name="numDims",
            description="num dims",
            reference="numDims"
        )
    ],
    trial_spec=trial_spec
)

In [8]:
from kubeflow.katib import KatibClient
from kubernetes.client import V1ObjectMeta
from kubeflow.katib import V1beta1Experiment
from kubeflow.katib import V1beta1AlgorithmSpec
from kubeflow.katib import V1beta1AlgorithmSetting
from kubeflow.katib import V1beta1ObjectiveSpec
from kubeflow.katib import V1beta1MetricsCollectorSpec
from kubeflow.katib import V1beta1CollectorSpec
from kubeflow.katib import V1beta1SourceSpec
from kubeflow.katib import V1beta1FilterSpec
from kubeflow.katib import V1beta1FeasibleSpace
from kubeflow.katib import V1beta1ExperimentSpec
from kubeflow.katib import V1beta1NasConfig
from kubeflow.katib import V1beta1GraphConfig
from kubeflow.katib import V1beta1Operation
from kubeflow.katib import V1beta1ParameterSpec
from kubeflow.katib import V1beta1TrialTemplate
from kubeflow.katib import V1beta1TrialParameterSpec
import copy


In [15]:
# experiment_spec=V1beta1ExperimentSpec(
#     max_trial_count=max_trial_count,
#     max_failed_trial_count=max_failed_trial_count,
#     parallel_trial_count=parallel_trial_count,
#     objective=objective,
#     algorithm=algorithm,
#     parameters=parameters,
#     trial_template=trial_template
# )

metadata = V1ObjectMeta(
    name="testing",
    namespace="kubeflow"
)

experiment = V1beta1Experiment(
    api_version="kubeflow.org/v1beta1",
    kind="Experiment",
    metadata = metadata,
    spec=V1beta1ExperimentSpec(
        max_trial_count=1,
        parallel_trial_count=1,
        max_failed_trial_count=1,
        algorithm=algorithm,
        objective=objective,
        trial_template=trial_template,
        parameters=parameters
    )
)

In [17]:
print(experiment.trial_template)


{'config_map': None,
 'failure_condition': None,
 'primary_container_name': 'training-container',
 'primary_pod_labels': None,
 'retain': True,
 'success_condition': None,
 'trial_parameters': [{'description': 'Learning rate for the training model',
                       'name': 'learningRate',
                       'reference': 'learningRate'},
                      {'description': 'Number of training model layers',
                       'name': 'numberLayers',
                       'reference': 'numberLayers'},
                      {'description': 'Training model optimizer (sdg, adam or '
                                      'ftrl)',
                       'name': 'optimizer',
                       'reference': 'optimizer'},
                      {'description': 'drop out',
                       'name': 'dropOut',
                       'reference': 'dropOut'},
                      {'description': 'batch size',
                       'name': 'batchSize',
                    

In [16]:
# Create client.
from kubeflow.katib import KatibClient
kclient = KatibClient()

# Create your Experiment.
kclient.create_experiment(experiment,namespace="kubeflow")

{'apiVersion': 'kubeflow.org/v1beta1',
 'kind': 'Experiment',
 'metadata': {'creationTimestamp': '2022-09-13T11:20:59Z',
  'generation': 1,
  'managedFields': [{'apiVersion': 'kubeflow.org/v1beta1',
    'fieldsType': 'FieldsV1',
    'fieldsV1': {'f:spec': {'.': {},
      'f:algorithm': {'.': {}, 'f:algorithmName': {}},
      'f:maxFailedTrialCount': {},
      'f:maxTrialCount': {},
      'f:objective': {'.': {},
       'f:additionalMetricNames': {},
       'f:goal': {},
       'f:objectiveMetricName': {},
       'f:type': {}},
      'f:parallelTrialCount': {},
      'f:parameters': {},
      'f:trialTemplate': {'.': {},
       'f:primaryContainerName': {},
       'f:retain': {},
       'f:trialParameters': {},
       'f:trialSpec': {'.': {},
        'f:apiVersion': {},
        'f:kind': {},
        'f:spec': {'.': {},
         'f:template': {'.': {},
          'f:metadata': {'.': {},
           'f:annotations': {'.': {}, 'f:sidecar.istio.io/inject': {}}},
          'f:spec': {'.': {},
